# São Paulo

In [ ]:
import os
import pandas as pd

pasta_downloads = r'C:\Users\ricardo\Downloads'
nome_arquivo_base = 'Restrições de Circulação - SP'

# Obter o caminho do arquivo mais recente que contém o nome base
arquivos = [os.path.join(pasta_downloads, arquivo) for arquivo in os.listdir(pasta_downloads)]
arquivos_restricoes_sp = [arquivo for arquivo in arquivos if nome_arquivo_base in arquivo]
arquivo_mais_recente = max(arquivos_restricoes_sp, key=os.path.getmtime)

# Carregar os dados do Excel usando pandas
dados_excel_base = pd.read_excel(arquivo_mais_recente, sheet_name="BASE")

# Filtrar os municípios com base na coluna "Usar"
municipios_usar = dados_excel_base.loc[dados_excel_base["Usar?"] == "Sim", "Município"].tolist()

# Mapeamento de cores em HTML
cores_html = {
    'Laranja': '#EF7D19',
    'Roxo': '#7A579C',
    'Vermelho': '#E30613'
    # Adicione mais cores conforme necessário
}

# Criar o início do script HTML
html_script = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Municípios</title>
  <style>
    table {
      margin-bottom: 20px;
    }
    th {
      background-color: #0C0D0E;
      color: #FFFFFF;
      font-weight: bold;
    }
    td.color-cell {
      width: 20px;
      height: 20px;
      text-align: center;
      vertical-align: middle;
    }
    td.bold-text {
      font-weight: bold;
    }
    #gif-seta {
      position: absolute;
      left: 10px;
      z-index: 9999; /* Garante que o gif esteja acima das tabelas */
      display: none; /* Começa oculto */
    }
    #mapaContainer {
      margin-top: 25px; /* Ajuste a altura do mapa para ficar 25px abaixo do gif */
    }
  </style>
</head>
<body>

  <label for="municipio">Selecione o Município:</label>
  <select id="municipio" onchange="atualizarConteudo()">
    <option value="">Selecione...</option>
"""

# Adicionar as opções dos municípios ao script HTML
for municipio in municipios_usar:
    html_script += f'    <option value="{municipio}">{municipio}</option>\n'

# Adicionar o restante do script HTML
html_script += """
  </select>

  <div id="tabelaContainer">
"""

# Adicionar as tabelas ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município
    dados_excel_municipio = pd.read_excel(arquivo_mais_recente, sheet_name=municipio)

    html_script += f'    <table id="tabela{municipio}" style="display: none;">\n'
    html_script += '      <thead>\n'
    html_script += '        <tr>\n'
    html_script += '          <th class="bold-text">FORMA DE RESTRIÇÃO</th>\n'
    html_script += '          <th class="bold-text">SEGUNDA - SEXTA</th>\n'
    html_script += '          <th class="bold-text">SÁBADO</th>\n'
    html_script += '          <th class="bold-text">DOMINGO - FERIADOS</th>\n'
    html_script += '        </tr>\n'
    html_script += '      </thead>\n'
    html_script += '      <tbody>\n'

    # Adicionar uma linha para cada entrada nas colunas
    for index, row in dados_excel_municipio.iterrows():
        cor_segunda = cores_html.get(row["COR SEGUNDA - SEXTA"], '')
        cor_sabado = cores_html.get(row["COR SÁBADO"], '')
        cor_domingo = cores_html.get(row["COR DOMINGO - FERIADOS"], '')

        # Verificar se as células nas colunas "SÁBADO" e "DOMINGO - FERIADOS" indicam mesclagem
        mesclar_sabado = row["SÁBADO"] == "mesclar"
        mesclar_domingo = row["DOMINGO - FERIADOS"] == "mesclar"

        html_script += '        <tr>'
        
        # Função para criar a célula com colspan
        def criar_celula_colspan(valor, colspan):
            return f'<td colspan="{colspan}" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{valor}</td>'
        
        if mesclar_sabado or mesclar_domingo:
            # Mesclar a célula "FORMA DE RESTRIÇÃO"
            html_script += f'<td colspan="1" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}; font-weight: bold; text-align: left;" class="color-cell">{row["FORMA DE RESTRIÇÃO"]}</td>'
            
            # Mesclar a célula "SEGUNDA - SEXTA"
            html_script += criar_celula_colspan(row["SEGUNDA - SEXTA"], 3)

            html_script += '</tr>\n'
        else:
            # Caso padrão sem mesclagem
            html_script += f'<td class="bold-text" style="font-weight: bold; font-size: 14px;">{row["FORMA DE RESTRIÇÃO"]}</td>' \
                f'<td style="font-size: 14px; background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{row["SEGUNDA - SEXTA"]}</td>' \
                f'<td style="font-size: 14px; background-color: {cor_sabado}; color: {"#FFFFFF" if cor_sabado else "#000000"}" class="color-cell">{row["SÁBADO"]}</td>' \
                f'<td style="font-size: 14px; background-color: {cor_domingo}; color: {"#FFFFFF" if cor_domingo else "#000000"}" class="color-cell">{row["DOMINGO - FERIADOS"]}</td></tr>\n'

    html_script += '        <!-- Adicione mais linhas conforme necessário -->\n'
    html_script += '      </tbody>\n'
    html_script += '    </table>\n'

# Adicionar o gif abaixo das tabelas
html_script += '<img id="gif-seta" src="http://www.iptcsp.com.br/wp-content/uploads/2024/02/gif-seta-1.gif" alt="Seta" style="width: 50px; height: 50px;">'

# Adicionar o texto abaixo das tabelas, a ser exibido apenas quando um município for selecionado
html_script += '<p id="textoInfoDetalhada" style="margin-top: -20px; display: none; left: 100px;">Clique aqui para acessar informações mais detalhadas sobre as restrições.</p>'

# Fechar a div de tabelas
html_script += """
  </div>

  <div id="mapaContainer">
"""

# Adicionar os mapas (iframes) ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município na aba "BASE"
    dados_excel_municipio_base = dados_excel_base[dados_excel_base["Município"] == municipio]
    
    html_script += f'    <div id="mapa{municipio}" style="display: none;">\n'
    html_script += f'      {dados_excel_municipio_base["Iframe"].values[0]}\n'
    html_script += '    </div>\n'

# Adicionar o restante do script HTML
html_script += """
  </div>

  <div id="infoExtrasContainer">
"""


# Adicionar as informações extras abaixo dos mapas
for municipio in municipios_usar:
    # Acessar a aba correspondente à informação extra no Excel
    dados_excel_municipio_info_extras = dados_excel_base[dados_excel_base["Município"] == municipio]
    if not dados_excel_municipio_info_extras.empty:
        # Abrir a div com margem
        html_script += f'    <div id="infoExtras{municipio}" style="display: none; margin: 20px; padding: 10px; border: 1px solid #ccc;">\n'
        
        # Obtendo os links das imagens
        link_eixos = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Eixos-com-legenda-e1713962942484.png"
        link_vuc = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Vuc-com-legenda-e1713962924634.png"
        link_peso = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Peso-com-legenda-e1713962906960.png"

        # Substituindo as palavras pelos ícones nas informações extras
        html_script += f'      <p><strong><img src="{link_eixos}" alt="EIXOS" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EIXOS"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_vuc}" alt="VUC" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["VUC"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_peso}" alt="PESO" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["PESO"].values[0]}</span></p>\n'

        # Adicionar informações de Orgão, Telefone, Email
        html_script += f'      <p><strong><span style="color: #0C0D0E;">Orgão:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["ORGÃO"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Telefone:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["TELEFONE"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Email:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EMAIL"].values[0]}</span></p>\n'
        
        # Adicionar informações de Ato normativo anterior, Ato normativo atual, Observação, Fonte web
        ato_normativo_anterior = dados_excel_municipio_info_extras["ATO NORMATIVO ANTERIOR"].values[0]
        ato_normativo = dados_excel_municipio_info_extras["ATO (s) NORMATIVO ATUAL"].values[0]
        observacao = dados_excel_municipio_info_extras["OBSERVAÇÃO"].values[0]
        fonte_web = dados_excel_municipio_info_extras["FONTE WEB"].values[0]

        # Verificar se os valores são NaN antes de adicioná-los ao script
        if pd.notna(ato_normativo_anterior):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo anterior:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo_anterior}</span></p>\n'

        if pd.notna(ato_normativo):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo atual:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo}</span></p>\n'

        if pd.notna(observacao):
            html_script += f'      <p><strong style="color: #0C0D0E;">Observação:</strong> ' \
                           f' <span style="color: #0C0D0E;">{observacao}</span></p>\n'

        if pd.notna(fonte_web):
            html_script += f'      <p><strong style="color: #0C0D0E;">Fonte Web:</strong> ' \
                           f' <span style="color: #0C0D0E;">{fonte_web}</span></p>\n'

        html_script += '    </div>\n'



# Fechar a div de informações extras
html_script += """
  </div>

  <script>
    function atualizarConteudo() {
      var selecionado = document.getElementById("municipio").value;

      // Exibir a tabela correspondente e ocultar as outras
      var tabelaContainer = document.getElementById("tabelaContainer");
      var tabelas = tabelaContainer.getElementsByTagName("table");
      for (var i = 0; i < tabelas.length; i++) {
        if (tabelas[i].id === "tabela" + selecionado) {
          tabelas[i].style.display = "table";
        } else {
          tabelas[i].style.display = "none";
        }
      }

      // Exibir o gif correspondente e ocultar os outros
      var gifSeta = document.getElementById("gif-seta");
      gifSeta.style.display = selecionado ? "block" : "none"; // Exibir o gif se um município estiver selecionado, caso contrário, ocultá-lo

      // Exibir o mapa correspondente e ocultar os outros
      var mapaContainer = document.getElementById("mapaContainer");
      var mapas = mapaContainer.getElementsByTagName("div");
      for (var i = 0; i < mapas.length; i++) {
        if (mapas[i].id === "mapa" + selecionado) {
          mapas[i].style.display = "block";
        } else {
          mapas[i].style.display = "none";
        }
      }

      // Exibir as informações extras correspondentes e ocultar as outras
      var infoExtrasContainer = document.getElementById("infoExtrasContainer");
      var infoExtras = infoExtrasContainer.getElementsByTagName("div");
      for (var i = 0; i < infoExtras.length; i++) {
        if (infoExtras[i].id === "infoExtras" + selecionado) {
          infoExtras[i].style.display = "block";
        } else {
          infoExtras[i].style.display = "none";
        }
      }

      // Exibir o texto "Clique aqui para acessar informações mais detalhadas sobre as restrições" se um município estiver selecionado, caso contrário, ocultá-lo
      var textoInfoDetalhada = document.getElementById("textoInfoDetalhada");
      textoInfoDetalhada.style.display = selecionado ? "block" : "none";
    }
  </script>

</body>
</html>
"""

# Imprimir o script HTML gerado
print(html_script)


# RIO DE JANEIRO

In [ ]:
import os
import pandas as pd

pasta_downloads = r'C:\Users\ricardo\Downloads'
nome_arquivo_base = 'Restrições de Circulação - RJ'

# Obter o caminho do arquivo mais recente que contém o nome base
arquivos = [os.path.join(pasta_downloads, arquivo) for arquivo in os.listdir(pasta_downloads)]
arquivos_restricoes_sp = [arquivo for arquivo in arquivos if nome_arquivo_base in arquivo]
arquivo_mais_recente = max(arquivos_restricoes_sp, key=os.path.getmtime)

# Carregar os dados do Excel usando pandas
dados_excel_base = pd.read_excel(arquivo_mais_recente, sheet_name="BASE")

# Filtrar os municípios com base na coluna "Usar"
municipios_usar = dados_excel_base.loc[dados_excel_base["Usar?"] == "Sim", "Município"].tolist()

# Mapeamento de cores em HTML
cores_html = {
    'Laranja': '#EF7D19',
    'Roxo': '#7A579C',
    'Vermelho': '#E30613'
    # Adicione mais cores conforme necessário
}

html_script = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Municípios</title>
  <style>
    table {
      margin-bottom: 20px;
    }
    th {
      background-color: #0C0D0E;
      color: #FFFFFF;
      font-weight: bold;
    }
    td.color-cell {
      width: 20px;
      height: 20px;
      text-align: center;
      vertical-align: middle;
    }
    td.bold-text {
      font-weight: bold;
    }
    #gif-seta {
      position: absolute;
      left: 10px;
      z-index: 9999; /* Garante que o gif esteja acima das tabelas */
      display: none; /* Começa oculto */
    }
    #mapaContainer {
      margin-top: 25px; /* Ajuste a altura do mapa para ficar 25px abaixo do gif */
    }
    #municipio {
      font-size: 20px; /* Tamanho da fonte dentro da caixa de seleção */
      width: 200px; /* Largura da caixa de seleção */
    }
    #municipio option {
      font-weight: bold;
      font-size: 18px; /* Tamanho da fonte das opções */
      color: black;
    }
    #municipio-label {
      font-size: 18px; /* Tamanho do texto do rótulo */
    }
  </style>
</head>
<body>

  <label for="municipio" id="municipio-label">Selecione o Município:</label>
  <select id="municipio" onchange="atualizarConteudo()">
    <option value="">Selecione...</option>
"""




# Adicionar as opções dos municípios ao script HTML
for municipio in municipios_usar:
    html_script += f'    <option value="{municipio}">{municipio}</option>\n'

# Adicionar o restante do script HTML
html_script += """
  </select>

  <div id="tabelaContainer">
"""

# Adicionar as tabelas ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município
    dados_excel_municipio = pd.read_excel(arquivo_mais_recente, sheet_name=municipio)

    html_script += f'    <table id="tabela{municipio}" style="display: none;">\n'
    html_script += '      <thead>\n'
    html_script += '        <tr>\n'
    html_script += '          <th class="bold-text">FORMA DE RESTRIÇÃO</th>\n'
    html_script += '          <th class="bold-text">SEGUNDA - SEXTA</th>\n'
    html_script += '          <th class="bold-text">SÁBADO</th>\n'
    html_script += '          <th class="bold-text">DOMINGO - FERIADOS</th>\n'
    html_script += '        </tr>\n'
    html_script += '      </thead>\n'
    html_script += '      <tbody>\n'

    for index, row in dados_excel_municipio.iterrows():
        cor_segunda = cores_html.get(row["COR SEGUNDA - SEXTA"], '')
        cor_sabado = cores_html.get(row["COR SÁBADO"], '')
        cor_domingo = cores_html.get(row["COR DOMINGO - FERIADOS"], '')

        # Verificar se as células nas colunas "SÁBADO" e "DOMINGO - FERIADOS" indicam mesclagem
        mesclar_sabado = row["SÁBADO"] == "mesclar"
        mesclar_domingo = row["DOMINGO - FERIADOS"] == "mesclar"

        html_script += '        <tr>'

        # Função para criar a célula com colspan
        def criar_celula_colspan(valor, colspan):
            return f'<td colspan="{colspan}" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{valor}</td>'

        if mesclar_sabado or mesclar_domingo:
            # Mesclar a célula "FORMA DE RESTRIÇÃO"
            html_script += f'<td colspan="1" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}; font-weight: bold; text-align: left;" class="color-cell">{row["FORMA DE RESTRIÇÃO"]}</td>'

            # Mesclar a célula "SEGUNDA - SEXTA"
            html_script += criar_celula_colspan(row["SEGUNDA - SEXTA"], 3)

            html_script += '</tr>\n'
        else:
            # Caso padrão sem mesclagem
            html_script += f'<td style="color: black;" class="bold-text">{row["FORMA DE RESTRIÇÃO"]}</td>' \
                            f'<td style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}; font-weight: bold; font-size: 120%" class="color-cell">{row["SEGUNDA - SEXTA"]}</td>' \
                            f'<td style="background-color: {cor_sabado}; color: {"#FFFFFF" if cor_sabado else "#000000"}; font-weight: bold; font-size: 120%" class="color-cell">{row["SÁBADO"]}</td>' \
                            f'<td style="background-color: {cor_domingo}; color: {"#FFFFFF" if cor_domingo else "#000000"}; font-weight: bold; font-size: 120%" class="color-cell">{row["DOMINGO - FERIADOS"]}</td></tr>\n'

    html_script += '        <!-- Adicione mais linhas conforme necessário -->\n'
    html_script += '      </tbody>\n'
    html_script += '    </table>\n'

# Adicionar o gif abaixo das tabelas
html_script += '<img id="gif-seta" src="http://www.iptcsp.com.br/wp-content/uploads/2024/02/gif-seta-1.gif" alt="Seta" style="width: 50px; height: 50px;">'

# Adicionar o texto abaixo das tabelas, a ser exibido apenas quando um município for selecionado
html_script += '<p id="textoInfoDetalhada" style="margin-top: -20px; display: none; left: 100px; color: red;">Clique aqui para acessar informações mais detalhadas sobre as restrições.</p>'

# Fechar a div de tabelas
html_script += """
  </div>

  <div id="mapaContainer">
"""

# Adicionar os mapas (iframes) ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município na aba "BASE"
    dados_excel_municipio_base = dados_excel_base[dados_excel_base["Município"] == municipio]
    
    html_script += f'    <div id="mapa{municipio}" style="display: none;">\n'
    html_script += f'      {dados_excel_municipio_base["Iframe"].values[0]}\n'
    html_script += '    </div>\n'

# Adicionar o restante do script HTML
html_script += """
  </div>

  <div id="infoExtrasContainer">
"""


# Adicionar as informações extras abaixo dos mapas
for municipio in municipios_usar:
    # Acessar a aba correspondente à informação extra no Excel
    dados_excel_municipio_info_extras = dados_excel_base[dados_excel_base["Município"] == municipio]
    if not dados_excel_municipio_info_extras.empty:
        # Abrir a div com margem
        html_script += f'    <div id="infoExtras{municipio}" style="display: none; margin: 20px; padding: 10px; border: 1px solid #ccc;">\n'
        
        # Obtendo os links das imagens
        link_eixos = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Eixos-com-legenda-e1713962942484.png"
        link_vuc = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Vuc-com-legenda-e1713962924634.png"
        link_peso = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Peso-com-legenda-e1713962906960.png"

        # Substituindo as palavras pelos ícones nas informações extras
        html_script += f'      <p><strong><img src="{link_eixos}" alt="EIXOS" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EIXOS"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_vuc}" alt="VUC" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["VUC"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_peso}" alt="PESO" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["PESO"].values[0]}</span></p>\n'

        # Adicionar informações de Orgão, Telefone, Email
        html_script += f'      <p><strong><span style="color: #0C0D0E;">Orgão:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["ORGÃO"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Telefone:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["TELEFONE"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Email:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EMAIL"].values[0]}</span></p>\n'
        
        # Adicionar informações de Ato normativo anterior, Ato normativo atual, Observação, Fonte web
        ato_normativo_anterior = dados_excel_municipio_info_extras["ATO NORMATIVO ANTERIOR"].values[0]
        ato_normativo = dados_excel_municipio_info_extras["ATO (s) NORMATIVO ATUAL"].values[0]
        observacao = dados_excel_municipio_info_extras["OBSERVAÇÃO"].values[0]
        fonte_web = dados_excel_municipio_info_extras["FONTE WEB"].values[0]

        # Verificar se os valores são NaN antes de adicioná-los ao script
        if pd.notna(ato_normativo_anterior):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo anterior:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo_anterior}</span></p>\n'

        if pd.notna(ato_normativo):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo atual:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo}</span></p>\n'

        if pd.notna(observacao):
            html_script += f'      <p><strong style="color: #0C0D0E;">Observação:</strong> ' \
                           f' <span style="color: #0C0D0E;">{observacao}</span></p>\n'

        if pd.notna(fonte_web):
            html_script += f'      <p><strong style="color: #0C0D0E;">Fonte Web:</strong> ' \
                           f' <span style="color: #0C0D0E;">{fonte_web}</span></p>\n'

        html_script += '    </div>\n'



# Fechar a div de informações extras
html_script += """
  </div>

  <script>
    function atualizarConteudo() {
      var selecionado = document.getElementById("municipio").value;

      // Exibir a tabela correspondente e ocultar as outras
      var tabelaContainer = document.getElementById("tabelaContainer");
      var tabelas = tabelaContainer.getElementsByTagName("table");
      for (var i = 0; i < tabelas.length; i++) {
        if (tabelas[i].id === "tabela" + selecionado) {
          tabelas[i].style.display = "table";
        } else {
          tabelas[i].style.display = "none";
        }
      }

      // Exibir o gif correspondente e ocultar os outros
      var gifSeta = document.getElementById("gif-seta");
      gifSeta.style.display = selecionado ? "block" : "none"; // Exibir o gif se um município estiver selecionado, caso contrário, ocultá-lo

      // Exibir o mapa correspondente e ocultar os outros
      var mapaContainer = document.getElementById("mapaContainer");
      var mapas = mapaContainer.getElementsByTagName("div");
      for (var i = 0; i < mapas.length; i++) {
        if (mapas[i].id === "mapa" + selecionado) {
          mapas[i].style.display = "block";
        } else {
          mapas[i].style.display = "none";
        }
      }

      // Exibir as informações extras correspondentes e ocultar as outras
      var infoExtrasContainer = document.getElementById("infoExtrasContainer");
      var infoExtras = infoExtrasContainer.getElementsByTagName("div");
      for (var i = 0; i < infoExtras.length; i++) {
        if (infoExtras[i].id === "infoExtras" + selecionado) {
          infoExtras[i].style.display = "block";
        } else {
          infoExtras[i].style.display = "none";
        }
      }

      // Exibir o texto "Clique aqui para acessar informações mais detalhadas sobre as restrições" se um município estiver selecionado, caso contrário, ocultá-lo
      var textoInfoDetalhada = document.getElementById("textoInfoDetalhada");
      textoInfoDetalhada.style.display = selecionado ? "block" : "none";
    }
  </script>

</body>
</html>
"""

# Imprimir o script HTML gerado
print(html_script)


# MINAS GERAIS

In [ ]:
import os
import pandas as pd

pasta_downloads = r'C:\Users\ricardo\Downloads'
nome_arquivo_base = 'Restrições de Circulação - MG'

# Obter o caminho do arquivo mais recente que contém o nome base
arquivos = [os.path.join(pasta_downloads, arquivo) for arquivo in os.listdir(pasta_downloads)]
arquivos_restricoes_sp = [arquivo for arquivo in arquivos if nome_arquivo_base in arquivo]
arquivo_mais_recente = max(arquivos_restricoes_sp, key=os.path.getmtime)

# Carregar os dados do Excel usando pandas
dados_excel_base = pd.read_excel(arquivo_mais_recente, sheet_name="BASE")

# Filtrar os municípios com base na coluna "Usar"
municipios_usar = dados_excel_base.loc[dados_excel_base["Usar?"] == "Sim", "Município"].tolist()

# Mapeamento de cores em HTML
cores_html = {
    'Laranja': '#EF7D19',
    'Roxo': '#7A579C',
    'Vermelho': '#E30613'
    # Adicione mais cores conforme necessário
}

# Criar o início do script HTML
html_script = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Municípios</title>
  <style>
    table {
      margin-bottom: 20px;
    }
    th {
      background-color: #0C0D0E;
      color: #FFFFFF;
      font-weight: bold;
    }
    td.color-cell {
      width: 20px;
      height: 20px;
      text-align: center;
      vertical-align: middle;
    }
    td.bold-text {
      font-weight: bold;
    }
    #gif-seta {
      position: absolute;
      left: 10px;
      z-index: 9999; /* Garante que o gif esteja acima das tabelas */
      display: none; /* Começa oculto */
    }
    #mapaContainer {
      margin-top: 25px; /* Ajuste a altura do mapa para ficar 25px abaixo do gif */
    }
  </style>
</head>
<body>

  <label for="municipio">Selecione o Município:</label>
  <select id="municipio" onchange="atualizarConteudo()">
    <option value="">Selecione...</option>
"""

# Adicionar as opções dos municípios ao script HTML
for municipio in municipios_usar:
    html_script += f'    <option value="{municipio}">{municipio}</option>\n'

# Adicionar o restante do script HTML
html_script += """
  </select>

  <div id="tabelaContainer">
"""

# Adicionar as tabelas ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município
    dados_excel_municipio = pd.read_excel(arquivo_mais_recente, sheet_name=municipio)

    html_script += f'    <table id="tabela{municipio}" style="display: none;">\n'
    html_script += '      <thead>\n'
    html_script += '        <tr>\n'
    html_script += '          <th class="bold-text">FORMA DE RESTRIÇÃO</th>\n'
    html_script += '          <th class="bold-text">SEGUNDA - SEXTA</th>\n'
    html_script += '          <th class="bold-text">SÁBADO</th>\n'
    html_script += '          <th class="bold-text">DOMINGO - FERIADOS</th>\n'
    html_script += '        </tr>\n'
    html_script += '      </thead>\n'
    html_script += '      <tbody>\n'

    # Adicionar uma linha para cada entrada nas colunas
    for index, row in dados_excel_municipio.iterrows():
        cor_segunda = cores_html.get(row["COR SEGUNDA - SEXTA"], '')
        cor_sabado = cores_html.get(row["COR SÁBADO"], '')
        cor_domingo = cores_html.get(row["COR DOMINGO - FERIADOS"], '')

        # Verificar se as células nas colunas "SÁBADO" e "DOMINGO - FERIADOS" indicam mesclagem
        mesclar_sabado = row["SÁBADO"] == "mesclar"
        mesclar_domingo = row["DOMINGO - FERIADOS"] == "mesclar"

        html_script += '        <tr>'
        
        # Função para criar a célula com colspan
        def criar_celula_colspan(valor, colspan):
            return f'<td colspan="{colspan}" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{valor}</td>'
        
        if mesclar_sabado or mesclar_domingo:
            # Mesclar a célula "FORMA DE RESTRIÇÃO"
            html_script += f'<td colspan="1" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}; font-weight: bold; text-align: left;" class="color-cell">{row["FORMA DE RESTRIÇÃO"]}</td>'
            
            # Mesclar a célula "SEGUNDA - SEXTA"
            html_script += criar_celula_colspan(row["SEGUNDA - SEXTA"], 3)

            html_script += '</tr>\n'
        else:
            # Caso padrão sem mesclagem
            html_script += f'<td class="bold-text">{row["FORMA DE RESTRIÇÃO"]}</td>' \
                            f'<td style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{row["SEGUNDA - SEXTA"]}</td>' \
                            f'<td style="background-color: {cor_sabado}; color: {"#FFFFFF" if cor_sabado else "#000000"}" class="color-cell">{row["SÁBADO"]}</td>' \
                            f'<td style="background-color: {cor_domingo}; color: {"#FFFFFF" if cor_domingo else "#000000"}" class="color-cell">{row["DOMINGO - FERIADOS"]}</td></tr>\n'

    html_script += '        <!-- Adicione mais linhas conforme necessário -->\n'
    html_script += '      </tbody>\n'
    html_script += '    </table>\n'

# Adicionar o gif abaixo das tabelas
html_script += '<img id="gif-seta" src="http://www.iptcsp.com.br/wp-content/uploads/2024/02/gif-seta-1.gif" alt="Seta" style="width: 50px; height: 50px;">'

# Adicionar o texto abaixo das tabelas, a ser exibido apenas quando um município for selecionado
html_script += '<p id="textoInfoDetalhada" style="margin-top: -20px; display: none; left: 100px;">Clique aqui para acessar informações mais detalhadas sobre as restrições.</p>'

# Fechar a div de tabelas
html_script += """
  </div>

  <div id="mapaContainer">
"""

# Adicionar os mapas (iframes) ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município na aba "BASE"
    dados_excel_municipio_base = dados_excel_base[dados_excel_base["Município"] == municipio]
    
    html_script += f'    <div id="mapa{municipio}" style="display: none;">\n'
    html_script += f'      {dados_excel_municipio_base["Iframe"].values[0]}\n'
    html_script += '    </div>\n'

# Adicionar o restante do script HTML
html_script += """
  </div>

  <div id="infoExtrasContainer">
"""


# Adicionar as informações extras abaixo dos mapas
for municipio in municipios_usar:
    # Acessar a aba correspondente à informação extra no Excel
    dados_excel_municipio_info_extras = dados_excel_base[dados_excel_base["Município"] == municipio]
    if not dados_excel_municipio_info_extras.empty:
        # Abrir a div com margem
        html_script += f'    <div id="infoExtras{municipio}" style="display: none; margin: 20px; padding: 10px; border: 1px solid #ccc;">\n'
        
        # Obtendo os links das imagens
        link_eixos = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Eixos-com-legenda-e1713962942484.png"
        link_vuc = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Vuc-com-legenda-e1713962924634.png"
        link_peso = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Peso-com-legenda-e1713962906960.png"

        # Substituindo as palavras pelos ícones nas informações extras
        html_script += f'      <p><strong><img src="{link_eixos}" alt="EIXOS" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EIXOS"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_vuc}" alt="VUC" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["VUC"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_peso}" alt="PESO" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["PESO"].values[0]}</span></p>\n'

        # Adicionar informações de Orgão, Telefone, Email
        html_script += f'      <p><strong><span style="color: #0C0D0E;">Orgão:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["ORGÃO"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Telefone:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["TELEFONE"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Email:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EMAIL"].values[0]}</span></p>\n'
        
        # Adicionar informações de Ato normativo anterior, Ato normativo atual, Observação, Fonte web
        ato_normativo_anterior = dados_excel_municipio_info_extras["ATO NORMATIVO ANTERIOR"].values[0]
        ato_normativo = dados_excel_municipio_info_extras["ATO (s) NORMATIVO ATUAL"].values[0]
        observacao = dados_excel_municipio_info_extras["OBSERVAÇÃO"].values[0]
        fonte_web = dados_excel_municipio_info_extras["FONTE WEB"].values[0]

        # Verificar se os valores são NaN antes de adicioná-los ao script
        if pd.notna(ato_normativo_anterior):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo anterior:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo_anterior}</span></p>\n'

        if pd.notna(ato_normativo):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo atual:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo}</span></p>\n'

        if pd.notna(observacao):
            html_script += f'      <p><strong style="color: #0C0D0E;">Observação:</strong> ' \
                           f' <span style="color: #0C0D0E;">{observacao}</span></p>\n'

        if pd.notna(fonte_web):
            html_script += f'      <p><strong style="color: #0C0D0E;">Fonte Web:</strong> ' \
                           f' <span style="color: #0C0D0E;">{fonte_web}</span></p>\n'

        html_script += '    </div>\n'



# Fechar a div de informações extras
html_script += """
  </div>

  <script>
    function atualizarConteudo() {
      var selecionado = document.getElementById("municipio").value;

      // Exibir a tabela correspondente e ocultar as outras
      var tabelaContainer = document.getElementById("tabelaContainer");
      var tabelas = tabelaContainer.getElementsByTagName("table");
      for (var i = 0; i < tabelas.length; i++) {
        if (tabelas[i].id === "tabela" + selecionado) {
          tabelas[i].style.display = "table";
        } else {
          tabelas[i].style.display = "none";
        }
      }

      // Exibir o gif correspondente e ocultar os outros
      var gifSeta = document.getElementById("gif-seta");
      gifSeta.style.display = selecionado ? "block" : "none"; // Exibir o gif se um município estiver selecionado, caso contrário, ocultá-lo

      // Exibir o mapa correspondente e ocultar os outros
      var mapaContainer = document.getElementById("mapaContainer");
      var mapas = mapaContainer.getElementsByTagName("div");
      for (var i = 0; i < mapas.length; i++) {
        if (mapas[i].id === "mapa" + selecionado) {
          mapas[i].style.display = "block";
        } else {
          mapas[i].style.display = "none";
        }
      }

      // Exibir as informações extras correspondentes e ocultar as outras
      var infoExtrasContainer = document.getElementById("infoExtrasContainer");
      var infoExtras = infoExtrasContainer.getElementsByTagName("div");
      for (var i = 0; i < infoExtras.length; i++) {
        if (infoExtras[i].id === "infoExtras" + selecionado) {
          infoExtras[i].style.display = "block";
        } else {
          infoExtras[i].style.display = "none";
        }
      }

      // Exibir o texto "Clique aqui para acessar informações mais detalhadas sobre as restrições" se um município estiver selecionado, caso contrário, ocultá-lo
      var textoInfoDetalhada = document.getElementById("textoInfoDetalhada");
      textoInfoDetalhada.style.display = selecionado ? "block" : "none";
    }
  </script>

</body>
</html>
"""

# Imprimir o script HTML gerado
print(html_script)


# ESPÍRITO SANTO

In [ ]:
import os
import pandas as pd

pasta_downloads = r'C:\Users\ricardo\Downloads'
nome_arquivo_base = 'Restrições de Circulação - ES'

# Obter o caminho do arquivo mais recente que contém o nome base
arquivos = [os.path.join(pasta_downloads, arquivo) for arquivo in os.listdir(pasta_downloads)]
arquivos_restricoes_sp = [arquivo for arquivo in arquivos if nome_arquivo_base in arquivo]
arquivo_mais_recente = max(arquivos_restricoes_sp, key=os.path.getmtime)

# Carregar os dados do Excel usando pandas
dados_excel_base = pd.read_excel(arquivo_mais_recente, sheet_name="BASE")

# Filtrar os municípios com base na coluna "Usar"
municipios_usar = dados_excel_base.loc[dados_excel_base["Usar?"] == "Sim", "Município"].tolist()

# Mapeamento de cores em HTML
cores_html = {
    'Laranja': '#EF7D19',
    'Roxo': '#7A579C',
    'Vermelho': '#E30613'
    # Adicione mais cores conforme necessário
}

# Criar o início do script HTML
html_script = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Municípios</title>
  <style>
    table {
      margin-bottom: 20px;
    }
    th {
      background-color: #0C0D0E;
      color: #FFFFFF;
      font-weight: bold;
    }
    td.color-cell {
      width: 20px;
      height: 20px;
      text-align: center;
      vertical-align: middle;
    }
    td.bold-text {
      font-weight: bold;
    }
    #gif-seta {
      position: absolute;
      left: 10px;
      z-index: 9999; /* Garante que o gif esteja acima das tabelas */
      display: none; /* Começa oculto */
    }
    #mapaContainer {
      margin-top: 25px; /* Ajuste a altura do mapa para ficar 25px abaixo do gif */
    }
  </style>
</head>
<body>

  <label for="municipio">Selecione o Município:</label>
  <select id="municipio" onchange="atualizarConteudo()">
    <option value="">Selecione...</option>
"""

# Adicionar as opções dos municípios ao script HTML
for municipio in municipios_usar:
    html_script += f'    <option value="{municipio}">{municipio}</option>\n'

# Adicionar o restante do script HTML
html_script += """
  </select>

  <div id="tabelaContainer">
"""

# Adicionar as tabelas ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município
    dados_excel_municipio = pd.read_excel(arquivo_mais_recente, sheet_name=municipio)

    html_script += f'    <table id="tabela{municipio}" style="display: none;">\n'
    html_script += '      <thead>\n'
    html_script += '        <tr>\n'
    html_script += '          <th class="bold-text">FORMA DE RESTRIÇÃO</th>\n'
    html_script += '          <th class="bold-text">SEGUNDA - SEXTA</th>\n'
    html_script += '          <th class="bold-text">SÁBADO</th>\n'
    html_script += '          <th class="bold-text">DOMINGO - FERIADOS</th>\n'
    html_script += '        </tr>\n'
    html_script += '      </thead>\n'
    html_script += '      <tbody>\n'

    # Adicionar uma linha para cada entrada nas colunas
    for index, row in dados_excel_municipio.iterrows():
        cor_segunda = cores_html.get(row["COR SEGUNDA - SEXTA"], '')
        cor_sabado = cores_html.get(row["COR SÁBADO"], '')
        cor_domingo = cores_html.get(row["COR DOMINGO - FERIADOS"], '')

        # Verificar se as células nas colunas "SÁBADO" e "DOMINGO - FERIADOS" indicam mesclagem
        mesclar_sabado = row["SÁBADO"] == "mesclar"
        mesclar_domingo = row["DOMINGO - FERIADOS"] == "mesclar"

        html_script += '        <tr>'
        
        # Função para criar a célula com colspan
        def criar_celula_colspan(valor, colspan):
            return f'<td colspan="{colspan}" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{valor}</td>'
        
        if mesclar_sabado or mesclar_domingo:
            # Mesclar a célula "FORMA DE RESTRIÇÃO"
            html_script += f'<td colspan="1" style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}; font-weight: bold; text-align: left;" class="color-cell">{row["FORMA DE RESTRIÇÃO"]}</td>'
            
            # Mesclar a célula "SEGUNDA - SEXTA"
            html_script += criar_celula_colspan(row["SEGUNDA - SEXTA"], 3)

            html_script += '</tr>\n'
        else:
            # Caso padrão sem mesclagem
            html_script += f'<td class="bold-text">{row["FORMA DE RESTRIÇÃO"]}</td>' \
                            f'<td style="background-color: {cor_segunda}; color: {"#FFFFFF" if cor_segunda else "#000000"}" class="color-cell">{row["SEGUNDA - SEXTA"]}</td>' \
                            f'<td style="background-color: {cor_sabado}; color: {"#FFFFFF" if cor_sabado else "#000000"}" class="color-cell">{row["SÁBADO"]}</td>' \
                            f'<td style="background-color: {cor_domingo}; color: {"#FFFFFF" if cor_domingo else "#000000"}" class="color-cell">{row["DOMINGO - FERIADOS"]}</td></tr>\n'

    html_script += '        <!-- Adicione mais linhas conforme necessário -->\n'
    html_script += '      </tbody>\n'
    html_script += '    </table>\n'

# Adicionar o gif abaixo das tabelas
html_script += '<img id="gif-seta" src="http://www.iptcsp.com.br/wp-content/uploads/2024/02/gif-seta-1.gif" alt="Seta" style="width: 50px; height: 50px;">'

# Adicionar o texto abaixo das tabelas, a ser exibido apenas quando um município for selecionado
html_script += '<p id="textoInfoDetalhada" style="margin-top: -20px; display: none; left: 100px;">Clique aqui para acessar informações mais detalhadas sobre as restrições.</p>'

# Fechar a div de tabelas
html_script += """
  </div>

  <div id="mapaContainer">
"""

# Adicionar os mapas (iframes) ao script HTML
for municipio in municipios_usar:
    # Acessar a aba correspondente ao município na aba "BASE"
    dados_excel_municipio_base = dados_excel_base[dados_excel_base["Município"] == municipio]
    
    html_script += f'    <div id="mapa{municipio}" style="display: none;">\n'
    html_script += f'      {dados_excel_municipio_base["Iframe"].values[0]}\n'
    html_script += '    </div>\n'

# Adicionar o restante do script HTML
html_script += """
  </div>

  <div id="infoExtrasContainer">
"""


# Adicionar as informações extras abaixo dos mapas
for municipio in municipios_usar:
    # Acessar a aba correspondente à informação extra no Excel
    dados_excel_municipio_info_extras = dados_excel_base[dados_excel_base["Município"] == municipio]
    if not dados_excel_municipio_info_extras.empty:
        # Abrir a div com margem
        html_script += f'    <div id="infoExtras{municipio}" style="display: none; margin: 20px; padding: 10px; border: 1px solid #ccc;">\n'
        
        # Obtendo os links das imagens
        link_eixos = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Eixos-com-legenda-e1713962942484.png"
        link_vuc = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Vuc-com-legenda-e1713962924634.png"
        link_peso = "http://www.iptcsp.com.br/wp-content/uploads/2024/04/Peso-com-legenda-e1713962906960.png"

        # Substituindo as palavras pelos ícones nas informações extras
        html_script += f'      <p><strong><img src="{link_eixos}" alt="EIXOS" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EIXOS"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_vuc}" alt="VUC" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["VUC"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><img src="{link_peso}" alt="PESO" style="width: 7%;"></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["PESO"].values[0]}</span></p>\n'

        # Adicionar informações de Orgão, Telefone, Email
        html_script += f'      <p><strong><span style="color: #0C0D0E;">Orgão:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["ORGÃO"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Telefone:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["TELEFONE"].values[0]}</span></p>\n'

        html_script += f'      <p><strong><span style="color: #0C0D0E;">Email:</span></strong> ' \
                       f'<span style="color: #0C0D0E;">{dados_excel_municipio_info_extras["EMAIL"].values[0]}</span></p>\n'
        
        # Adicionar informações de Ato normativo anterior, Ato normativo atual, Observação, Fonte web
        ato_normativo_anterior = dados_excel_municipio_info_extras["ATO NORMATIVO ANTERIOR"].values[0]
        ato_normativo = dados_excel_municipio_info_extras["ATO (s) NORMATIVO ATUAL"].values[0]
        observacao = dados_excel_municipio_info_extras["OBSERVAÇÃO"].values[0]
        fonte_web = dados_excel_municipio_info_extras["FONTE WEB"].values[0]

        # Verificar se os valores são NaN antes de adicioná-los ao script
        if pd.notna(ato_normativo_anterior):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo anterior:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo_anterior}</span></p>\n'

        if pd.notna(ato_normativo):
            html_script += f'      <p><strong style="color: #0C0D0E;">Ato normativo atual:</strong> ' \
                           f' <span style="color: #0C0D0E;">{ato_normativo}</span></p>\n'

        if pd.notna(observacao):
            html_script += f'      <p><strong style="color: #0C0D0E;">Observação:</strong> ' \
                           f' <span style="color: #0C0D0E;">{observacao}</span></p>\n'

        if pd.notna(fonte_web):
            html_script += f'      <p><strong style="color: #0C0D0E;">Fonte Web:</strong> ' \
                           f' <span style="color: #0C0D0E;">{fonte_web}</span></p>\n'

        html_script += '    </div>\n'



# Fechar a div de informações extras
html_script += """
  </div>

  <script>
    function atualizarConteudo() {
      var selecionado = document.getElementById("municipio").value;

      // Exibir a tabela correspondente e ocultar as outras
      var tabelaContainer = document.getElementById("tabelaContainer");
      var tabelas = tabelaContainer.getElementsByTagName("table");
      for (var i = 0; i < tabelas.length; i++) {
        if (tabelas[i].id === "tabela" + selecionado) {
          tabelas[i].style.display = "table";
        } else {
          tabelas[i].style.display = "none";
        }
      }

      // Exibir o gif correspondente e ocultar os outros
      var gifSeta = document.getElementById("gif-seta");
      gifSeta.style.display = selecionado ? "block" : "none"; // Exibir o gif se um município estiver selecionado, caso contrário, ocultá-lo

      // Exibir o mapa correspondente e ocultar os outros
      var mapaContainer = document.getElementById("mapaContainer");
      var mapas = mapaContainer.getElementsByTagName("div");
      for (var i = 0; i < mapas.length; i++) {
        if (mapas[i].id === "mapa" + selecionado) {
          mapas[i].style.display = "block";
        } else {
          mapas[i].style.display = "none";
        }
      }

      // Exibir as informações extras correspondentes e ocultar as outras
      var infoExtrasContainer = document.getElementById("infoExtrasContainer");
      var infoExtras = infoExtrasContainer.getElementsByTagName("div");
      for (var i = 0; i < infoExtras.length; i++) {
        if (infoExtras[i].id === "infoExtras" + selecionado) {
          infoExtras[i].style.display = "block";
        } else {
          infoExtras[i].style.display = "none";
        }
      }

      // Exibir o texto "Clique aqui para acessar informações mais detalhadas sobre as restrições" se um município estiver selecionado, caso contrário, ocultá-lo
      var textoInfoDetalhada = document.getElementById("textoInfoDetalhada");
      textoInfoDetalhada.style.display = selecionado ? "block" : "none";
    }
  </script>

</body>
</html>
"""

# Imprimir o script HTML gerado
print(html_script)
